Resposta 3

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

# --- 1. Carregamento do Dataset Boston House-price 
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

X = data
y = target
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

df = pd.DataFrame(X, columns=feature_names)
df['MEDV'] = y

print("Dataset Boston House-price carregado com sucesso a partir de URL externa.")
print(f"\nShape do Dataset: {df.shape}")
print("Primeiras 5 linhas do Dataset:")
print(df.head())

# --- 2. Pré-processamento

# Estratégia de pré-processamento: Padronização (StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # Aplica o escalonamento aos dados de entrada

print("\nEtapa de Pré-processamento: Padronização (StandardScaler) aplicada aos atributos de entrada.")

# --- 3. Configuração dos Modelos 

# Regressor Linear
linear_reg_model = LinearRegression()

# MLP Regressor (Multi-layer Perceptron)
mlp_reg_model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                             max_iter=1000, random_state=42, early_stopping=True, n_iter_no_change=10)

print("\nModelos configurados: Regressor Linear e MLP Regressor.")
print(f"Parâmetros do MLPRegressor: {mlp_reg_model.get_params()}")

# --- 4. Avaliação com Validação Cruzada (10-fold Cross-Validation)

n_splits = 10 
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # Shuffle para embaralhar os dados

mae_linear_reg = []
mae_mlp_reg = []

print(f"\nIniciando Validação Cruzada ({n_splits}-fold)...")

for fold, (train_index, test_index) in enumerate(kf.split(X_scaled)):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Treinamento e Avaliação do Regressor Linear
    linear_reg_model.fit(X_train, y_train)
    y_pred_linear = linear_reg_model.predict(X_test)
    mae_linear_reg.append(mean_absolute_error(y_test, y_pred_linear))

    # Treinamento e Avaliação do MLP Regressor
    mlp_reg_model.fit(X_train, y_train)
    y_pred_mlp = mlp_reg_model.predict(X_test)
    mae_mlp_reg.append(mean_absolute_error(y_test, y_pred_mlp))

    print(f"Fold {fold+1}/{n_splits} - MAE Linear Reg: {mae_linear_reg[-1]:.4f}, MAE MLP Reg: {mae_mlp_reg[-1]:.4f}")

# Calcular a média dos MAEs para cada modelo
mean_mae_linear = np.mean(mae_linear_reg)
std_mae_linear = np.std(mae_linear_reg)

mean_mae_mlp = np.mean(mae_mlp_reg)
std_mae_mlp = np.std(mae_mlp_reg)

print("\n--- Resultados Finais da Validação Cruzada ---")
print(f"Regressor Linear: MAE Médio = {mean_mae_linear:.4f} (+/- {std_mae_linear:.4f})")
print(f"MLP Regressor: MAE Médio = {mean_mae_mlp:.4f} (+/- {std_mae_mlp:.4f})")

# --- 5. Obter o Modelo (Equação) do Melhor Regressor Linear 
if mean_mae_linear < mean_mae_mlp: # Se o Linear Regression for "melhor" (menor MAE)
    print("\n--- Modelo (Equação) do Regressor Linear (Melhor Desempenho) ---")
    linear_reg_model.fit(X_scaled, y)
    coefficients = linear_reg_model.coef_
    intercept = linear_reg_model.intercept_

    equation_parts = [f"{intercept:.4f}"]
    for i, coef in enumerate(coefficients):
        equation_parts.append(f" + ({coef:.4f} * {feature_names[i]})")

    print("Equação: MEDV =", "".join(equation_parts).replace(" + -", " - ")) # Ajusta a exibição de coeficientes negativos
else:
    print("\nO Regressor Linear não foi o modelo com o melhor desempenho médio nesta comparação.")

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Atlas Core\AppData\Local\Temp\ipykernel_17540\3688190145.py:11: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


Dataset Boston House-price carregado com sucesso a partir de URL externa.

Shape do Dataset: (506, 14)
Primeiras 5 linhas do Dataset:
      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  

Etapa de Pré-processamento: Padronização (StandardScaler) aplicada aos atributos de entrada.

Modelos configurados: Regressor Linear e MLP Regressor.
Parâmetros do MLPRegressor: {'activation': 'relu', 'alpha': 0.0001, 'bat

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

# --- 1. Carregamento do Dataset Boston House-price
data_url = "http://lib.stat.cmu.edu/datasets/boston" # URL corrigida
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

X = data
y = target
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

df = pd.DataFrame(X, columns=feature_names)
df['MEDV'] = y

# --- 2. Pré-processamento
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- 3. Configuração dos Modelos
linear_reg_model = LinearRegression()
mlp_reg_model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                             max_iter=1000, random_state=42, early_stopping=True, n_iter_no_change=10)

# --- 4. Avaliação com Validação Cruzada (10-fold Cross-Validation)
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mae_linear_reg = []
mae_mlp_reg = []

print("Iniciando Estudo Comparativo de Desempenho de Modelos (Boston House-Price Dataset)\n")
print(f"Dataset carregado. Shape: {df.shape}")
print("Primeiras 5 linhas do Dataset:\n", df.head().to_string()) # Usar .to_string() para melhor formatação

print("\nEtapa de Pré-processamento: Padronização (StandardScaler) aplicada aos atributos de entrada.")
print("Modelos configurados: Regressor Linear e MLP Regressor.")
print(f"Parâmetros do MLPRegressor: {mlp_reg_model.get_params()}")

print(f"\nIniciando Validação Cruzada ({n_splits}-fold)...")

for fold, (train_index, test_index) in enumerate(kf.split(X_scaled)):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    linear_reg_model.fit(X_train, y_train)
    y_pred_linear = linear_reg_model.predict(X_test)
    mae_linear_reg.append(mean_absolute_error(y_test, y_pred_linear))

    mlp_reg_model.fit(X_train, y_train)
    y_pred_mlp = mlp_reg_model.predict(X_test)
    mae_mlp_reg.append(mean_absolute_error(y_test, y_pred_mlp))

    print(f"  Fold {fold+1}/{n_splits} - MAE Linear Reg: {mae_linear_reg[-1]:.4f}, MAE MLP Reg: {mae_mlp_reg[-1]:.4f}")

# Calcular a média dos MAEs para cada modelo
mean_mae_linear = np.mean(mae_linear_reg)
std_mae_linear = np.std(mae_linear_reg)

mean_mae_mlp = np.mean(mae_mlp_reg)
std_mae_mlp = np.std(mae_mlp_reg)

# --- Formatar a Saída Final para Cópia Facilitada ---
output_str = "\n" + "="*70 + "\n"
output_str += " RESULTADOS FINAIS DO ESTUDO COMPARATIVO DE MODELOS \n"
output_str += "="*70 + "\n\n"

output_str += "1. Métricas de Desempenho (Erro Absoluto Médio - MAE):\n"
output_str += f"   - Regressor Linear: MAE Médio = {mean_mae_linear:.4f} (+/- {std_mae_linear:.4f})\n"
output_str += f"   - MLP Regressor:    MAE Médio = {mean_mae_mlp:.4f} (+/- {std_mae_mlp:.4f})\n\n"

# --- 5. Obter o Modelo (Equação) do Melhor Regressor Linear
if mean_mae_linear < mean_mae_mlp:
    output_str += "2. Modelo (Equação) do Regressor Linear (Melhor Desempenho):\n"
    linear_reg_model.fit(X_scaled, y) # Treinar em todo o dataset escalonado para obter a equação final
    coefficients = linear_reg_model.coef_
    intercept = linear_reg_model.intercept_

    equation_parts = [f"{intercept:.4f}"]
    for i, coef in enumerate(coefficients):
        equation_parts.append(f" + ({coef:.4f} * {feature_names[i]})")
    
    equation_str = "".join(equation_parts).replace(" + -", " - ")
    output_str += f"   Equação: MEDV = {equation_str}\n\n"
    output_str += "3. Discussão:\n"
    output_str += "   O Regressor Linear obteve um MAE médio menor, indicando um desempenho superior \n"
    output_str += "   neste estudo comparativo. Sua simplicidade e interpretabilidade são vantagens aqui.\n"
else:
    output_str += "2. Discussão dos Resultados:\n"
    output_str += "   O Regressor Linear não foi o modelo com o melhor desempenho médio nesta comparação.\n"
    output_str += f"   O MLP Regressor (MAE Médio: {mean_mae_mlp:.4f}) obteve um desempenho superior \n"
    output_str += f"   ao Regressor Linear (MAE Médio: {mean_mae_linear:.4f}).\n"
    output_str += "   Isso sugere que a MLP, com sua capacidade de aprender padrões não lineares, \n"
    output_str += "   foi mais eficaz em modelar as complexidades do dataset Boston House-Price. \n"
    output_str += "   A padronização dos dados foi crucial para o bom desempenho da MLP. \n"
    output_str += "   A escolha da arquitetura (camadas e neurônios) e dos hiperparâmetros \n"
    output_str += "   da MLP também influenciou significativamente seu resultado. \n"

output_str += "="*70 + "\n"
print(output_str)

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Atlas Core\AppData\Local\Temp\ipykernel_17540\4183854623.py:11: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


Iniciando Estudo Comparativo de Desempenho de Modelos (Boston House-Price Dataset)

Dataset carregado. Shape: (506, 14)
Primeiras 5 linhas do Dataset:
       CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO       B  LSTAT  MEDV
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0     15.3  396.90   4.98  24.0
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0     17.8  396.90   9.14  21.6
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0     17.8  392.83   4.03  34.7
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0     18.7  394.63   2.94  33.4
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0     18.7  396.90   5.33  36.2

Etapa de Pré-processamento: Padronização (StandardScaler) aplicada aos atributos de entrada.
Modelos configurados: Regressor Linear e MLP Regressor.
Parâmetros do MLPRegressor: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1'